# DUET Datasets EDA

Notebook này giúp bạn:
1) Scan nhiều polyp segmentation datasets theo layout thống nhất (codebase DUET).
2) Thống kê domain shift (color/texture/edge) giữa datasets, centers, modalities.
3) Phân tích small polyp và boundary difficulty (độ mảnh/độ phức tạp biên).

Yêu cầu: bạn đã tự tải dataset về máy và đặt đúng cấu trúc thư mục ở `DATA_ROOT`.


## 0. Setup

Cài thêm nếu thiếu:
```bash
pip install numpy pandas matplotlib scikit-learn scikit-image pillow tqdm
```


In [ ]:
from __future__ import annotations

import os
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from PIL import Image

from skimage.color import rgb2hsv
from skimage.feature import canny
from skimage.filters import sobel
from skimage.segmentation import find_boundaries

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

plt.rcParams['figure.dpi'] = 130

DATA_ROOT = Path(os.environ.get('DUET_DATA_ROOT', './data'))
print('DATA_ROOT =', DATA_ROOT.resolve())


## 1. Expected folder layout

Notebook (và codebase) đọc theo layout thống nhất, ví dụ:

```
data/
  kvasir_seg/
    images/
    masks/
  cvc_clinicdb/
    images/
    masks/
  cvc_colondb/
    images/
    masks/
  etis_larib/
    images/
    masks/
  polypdb/
    WLI/images/  WLI/annotations/
    NBI/images/  NBI/annotations/
    ...
    splits/
      train.txt
      test.txt
  polypgen/
    images/
      positive/
      negative/
    masks/
      positive/
    splits/
      loco_fold_1_train.txt
      loco_fold_1_test.txt
      ...
    meta/
      center_map.csv   # optional: image_id -> center_id
```

Chỉ cần đúng các folder `images/` và `masks/` là EDA chạy được; `splits/` giúp tách train/test/LOCO.


In [ ]:
@dataclass
class DatasetSpec:
    name: str
    images_glob: str
    masks_glob: Optional[str]  # None for unlabeled
    id_from_path: bool = True  # map by stem


SPECS: Dict[str, DatasetSpec] = {
    'kvasir_seg': DatasetSpec('kvasir_seg', 'kvasir_seg/images/*', 'kvasir_seg/masks/*'),
    'cvc_clinicdb': DatasetSpec('cvc_clinicdb', 'cvc_clinicdb/images/*', 'cvc_clinicdb/masks/*'),
    'cvc_colondb': DatasetSpec('cvc_colondb', 'cvc_colondb/images/*', 'cvc_colondb/masks/*'),
    'etis_larib': DatasetSpec('etis_larib', 'etis_larib/images/*', 'etis_larib/masks/*'),
    # PolypDB modality-wise layout (default from OSF wiki)
    'polypdb_WLI': DatasetSpec('polypdb_WLI', 'polypdb/WLI/images/*', 'polypdb/WLI/annotations/*'),
    'polypdb_NBI': DatasetSpec('polypdb_NBI', 'polypdb/NBI/images/*', 'polypdb/NBI/annotations/*'),
    'polypdb_BLI': DatasetSpec('polypdb_BLI', 'polypdb/BLI/images/*', 'polypdb/BLI/annotations/*'),
    'polypdb_LCI': DatasetSpec('polypdb_LCI', 'polypdb/LCI/images/*', 'polypdb/LCI/annotations/*'),
    'polypdb_FICE': DatasetSpec('polypdb_FICE', 'polypdb/FICE/images/*', 'polypdb/FICE/annotations/*'),
    # PolypGen positive/negative (unified layout)
    'polypgen_pos': DatasetSpec('polypgen_pos', 'polypgen/images/positive/*', 'polypgen/masks/positive/*'),
    'polypgen_neg': DatasetSpec('polypgen_neg', 'polypgen/images/negative/*', None),
}


def _list_files(pattern: str) -> List[Path]:
    return sorted([p for p in DATA_ROOT.glob(pattern) if p.is_file()])


def scan_dataset(spec: DatasetSpec) -> pd.DataFrame:
    imgs = _list_files(spec.images_glob)
    if len(imgs) == 0:
        return pd.DataFrame(columns=['dataset', 'image_path', 'mask_path', 'image_id'])

    if spec.masks_glob is None:
        df = pd.DataFrame({'dataset': spec.name, 'image_path': imgs})
        df['mask_path'] = None
        df['image_id'] = df['image_path'].apply(lambda p: Path(p).stem)
        return df

    masks = _list_files(spec.masks_glob)
    mask_map = {m.stem: m for m in masks}

    rows = []
    for im in imgs:
        iid = im.stem
        mp = mask_map.get(iid, None)
        # fallback: some datasets use different suffixes
        if mp is None:
            for k, v in mask_map.items():
                if k.startswith(iid) or iid.startswith(k):
                    mp = v
                    break
        rows.append({'dataset': spec.name, 'image_path': im, 'mask_path': mp, 'image_id': iid})

    df = pd.DataFrame(rows)
    return df


dfs = []
for key, spec in SPECS.items():
    df = scan_dataset(spec)
    if len(df) > 0:
        dfs.append(df)

data_index = pd.concat(dfs, ignore_index=True) if len(dfs) else pd.DataFrame()
print('Total samples:', len(data_index))
data_index['has_mask'] = data_index['mask_path'].notna()
display(data_index.groupby('dataset')[['image_id','has_mask']].count().rename(columns={'image_id':'n_images','has_mask':'n_with_mask'}))


## 2. Utility: load image/mask safely


In [ ]:
def read_rgb(path: Path) -> np.ndarray:
    img = Image.open(path).convert('RGB')
    return np.asarray(img)


def read_mask(path: Optional[Path]) -> Optional[np.ndarray]:
    if path is None or (isinstance(path, float) and np.isnan(path)):
        return None
    m = Image.open(path).convert('L')
    arr = np.asarray(m)
    # binarize (robust across 0/255 or 0/1)
    return (arr > 127).astype(np.uint8)


def mask_stats(mask: np.ndarray) -> Dict[str, float]:
    h, w = mask.shape
    area = float(mask.sum())
    area_ratio = area / float(h*w)

    bnd = find_boundaries(mask.astype(bool), mode='outer')
    bnd_len = float(bnd.sum())

    # boundary complexity proxy: boundary length per unit area (avoid div0)
    comp = bnd_len / (area + 1.0)
    bnd_ratio = bnd_len / float(h*w)
    return {'area': area, 'area_ratio': area_ratio, 'boundary_len': bnd_len, 'boundary_ratio': bnd_ratio, 'boundary_complexity': comp}


## 3. Basic dataset statistics

Gồm: resolution, area ratio, small polyp rate, boundary complexity.


In [ ]:
def compute_basic_stats(df: pd.DataFrame, max_items: Optional[int] = None) -> pd.DataFrame:
    rows = []
    it = df.itertuples(index=False)
    if max_items is not None:
        it = list(it)[:max_items]

    for r in tqdm(it, total=(max_items or len(df))):
        img = read_rgb(Path(r.image_path))
        h, w = img.shape[:2]
        row = {
            'dataset': r.dataset,
            'image_id': r.image_id,
            'H': h,
            'W': w,
        }
        m = read_mask(r.mask_path) if r.mask_path is not None else None
        if m is not None:
            row.update(mask_stats(m))
        rows.append(row)

    out = pd.DataFrame(rows)
    return out


stats = compute_basic_stats(data_index[data_index['has_mask']].copy(), max_items=None)
display(stats.head())

summary = stats.groupby('dataset').agg(
    n=('image_id','count'),
    H_mean=('H','mean'),
    W_mean=('W','mean'),
    area_ratio_median=('area_ratio','median'),
    area_ratio_p10=('area_ratio', lambda s: float(np.quantile(s, 0.10))),
    area_ratio_p90=('area_ratio', lambda s: float(np.quantile(s, 0.90))),
    boundary_complexity_median=('boundary_complexity','median'),
).reset_index()
display(summary)


### 3.1 Small polyp analysis

DUET nhấn mạnh nhóm small polyps (< 5mm). Nếu dataset không có size metadata, ta dùng proxy theo diện tích mask.

Bạn có thể chỉnh `SMALL_AREA_RATIO` tùy theo scale ảnh (ví dụ 0.005 tương đương 0.5% diện tích ảnh).


In [ ]:
SMALL_AREA_RATIO = 0.005

stats['is_small_proxy'] = stats['area_ratio'] < SMALL_AREA_RATIO
small_rate = stats.groupby('dataset')['is_small_proxy'].mean().reset_index().rename(columns={'is_small_proxy':'small_rate_proxy'})
display(small_rate)

plt.figure()
for ds, g in stats.groupby('dataset'):
    plt.hist(g['area_ratio'], bins=50, alpha=0.4, label=ds)
plt.xscale('log')
plt.xlabel('Mask area ratio (log-scale)')
plt.ylabel('Count')
plt.legend()
plt.title('Area ratio distribution by dataset')
plt.show()


### 3.2 Boundary difficulty

Một proxy đơn giản: `boundary_complexity = boundary_len/(area+1)`.
Giá trị cao thường là polyp nhỏ hoặc biên ngoằn ngoèo/mờ.


In [ ]:
plt.figure()
for ds, g in stats.groupby('dataset'):
    plt.hist(g['boundary_complexity'], bins=50, alpha=0.4, label=ds)
plt.yscale('log')
plt.xlabel('Boundary complexity (boundary_len/(area+1))')
plt.ylabel('Count (log)')
plt.legend()
plt.title('Boundary complexity distribution')
plt.show()


## 4. Visualize examples: overlay mask + boundary


In [ ]:
def show_overlay(img: np.ndarray, mask: np.ndarray, title: str = '') -> None:
    bnd = find_boundaries(mask.astype(bool), mode='outer')
    overlay = img.copy()
    # boundary in red
    overlay[bnd] = [255, 0, 0]

    plt.figure(figsize=(10,3))
    plt.subplot(1,3,1)
    plt.imshow(img)
    plt.axis('off')
    plt.title('Image')

    plt.subplot(1,3,2)
    plt.imshow(mask, cmap='gray')
    plt.axis('off')
    plt.title('Mask')

    plt.subplot(1,3,3)
    plt.imshow(overlay)
    plt.axis('off')
    plt.title('Overlay (boundary)')
    plt.suptitle(title)
    plt.show()


def sample_examples(df_stats: pd.DataFrame, n_per_ds: int = 3, mode: str = 'random') -> None:
    for ds, g in df_stats.groupby('dataset'):
        if len(g) == 0:
            continue
        if mode == 'small':
            cand = g[g['is_small_proxy']]
            if len(cand) == 0:
                continue
            samp = cand.sample(min(n_per_ds, len(cand)), random_state=0)
        elif mode == 'hard_boundary':
            samp = g.sort_values('boundary_complexity', ascending=False).head(n_per_ds)
        else:
            samp = g.sample(min(n_per_ds, len(g)), random_state=0)

        for r in samp.itertuples(index=False):
            img_path = data_index[(data_index['dataset']==ds) & (data_index['image_id']==r.image_id)].iloc[0]['image_path']
            mask_path = data_index[(data_index['dataset']==ds) & (data_index['image_id']==r.image_id)].iloc[0]['mask_path']
            img = read_rgb(Path(img_path))
            mask = read_mask(mask_path)
            show_overlay(img, mask, title=f'{ds} | id={r.image_id} | area_ratio={r.area_ratio:.4f} | bnd_comp={r.boundary_complexity:.3f}')


print('Random examples')
sample_examples(stats, n_per_ds=2, mode='random')

print('Small polyp proxy examples')
sample_examples(stats, n_per_ds=2, mode='small')

print('Hard boundary examples')
sample_examples(stats, n_per_ds=2, mode='hard_boundary')


## 5. Domain shift visualization (hand-crafted features)

Không cần pretrained model. Ta dùng feature đơn giản:
- mean/std RGB
- mean HSV
- edge density (Canny)
- gradient energy (Sobel)
- specular highlight ratio (V high, S low)


In [ ]:
def image_features(img: np.ndarray) -> np.ndarray:
    x = img.astype(np.float32) / 255.0
    h, w, _ = x.shape

    mean_rgb = x.reshape(-1, 3).mean(axis=0)
    std_rgb = x.reshape(-1, 3).std(axis=0)

    hsv = rgb2hsv(x)
    mean_hsv = hsv.reshape(-1, 3).mean(axis=0)

    gray = x.mean(axis=2)
    edges = canny(gray, sigma=1.0)
    edge_density = edges.mean()

    grad = sobel(gray)
    grad_energy = float(np.mean(grad**2))

    # specular: high V but low saturation
    spec = ((hsv[...,2] > 0.90) & (hsv[...,1] < 0.20)).mean()

    return np.concatenate([
        mean_rgb, std_rgb, mean_hsv,
        np.array([edge_density, grad_energy, spec], dtype=np.float32)
    ], axis=0)


def compute_features(df: pd.DataFrame, max_items_per_ds: int = 300) -> pd.DataFrame:
    rows = []
    for ds, g in df.groupby('dataset'):
        gg = g.sample(min(max_items_per_ds, len(g)), random_state=0)
        for r in tqdm(gg.itertuples(index=False), total=len(gg), desc=f'feat {ds}'):
            img = read_rgb(Path(r.image_path))
            feat = image_features(img)
            rows.append({'dataset': ds, 'image_id': r.image_id, 'feat': feat})

    out = pd.DataFrame(rows)
    feats = np.stack(out['feat'].values, axis=0)
    feat_cols = [
        'mean_r','mean_g','mean_b','std_r','std_g','std_b',
        'mean_h','mean_s','mean_v','edge_density','grad_energy','spec_ratio'
    ]
    out = pd.concat([out.drop(columns=['feat']).reset_index(drop=True), pd.DataFrame(feats, columns=feat_cols)], axis=1)
    return out


feat_df = compute_features(data_index.copy(), max_items_per_ds=300)
display(feat_df.head())


In [ ]:
X = feat_df.drop(columns=['dataset','image_id']).values

pca = PCA(n_components=2, random_state=0)
Z = pca.fit_transform(X)

plt.figure(figsize=(6,5))
for ds in sorted(feat_df['dataset'].unique()):
    m = feat_df['dataset'] == ds
    plt.scatter(Z[m,0], Z[m,1], s=10, alpha=0.6, label=ds)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('PCA on hand-crafted features (domain shift proxy)')
plt.legend(markerscale=2, bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# t-SNE thường chậm; giảm sample nếu cần
MAX_TSNE = 1200
ts = feat_df.sample(min(MAX_TSNE, len(feat_df)), random_state=0)
Xts = ts.drop(columns=['dataset','image_id']).values

tsne = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca', random_state=0)
Zts = tsne.fit_transform(Xts)

plt.figure(figsize=(6,5))
for ds in sorted(ts['dataset'].unique()):
    m = ts['dataset'] == ds
    plt.scatter(Zts[m,0], Zts[m,1], s=10, alpha=0.6, label=ds)
plt.title('t-SNE on hand-crafted features')
plt.legend(markerscale=2, bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


## 6. Boundary contrast proxy (tại sao biên mờ gây khó)

Một proxy: chênh lệch gradient magnitude giữa boundary band và background band.
Giá trị thấp nghĩa là biên khó phân tách (low-contrast boundary).


In [ ]:
from skimage.morphology import binary_dilation, binary_erosion, disk

def boundary_contrast_proxy(img: np.ndarray, mask: np.ndarray, r: int = 3) -> float:
    x = img.astype(np.float32) / 255.0
    gray = x.mean(axis=2)
    grad = sobel(gray)

    se = disk(r)
    core = binary_erosion(mask.astype(bool), se)
    dil = binary_dilation(mask.astype(bool), se)
    bnd_band = dil & (~core)
    bg_band = binary_dilation(dil, se) & (~dil)

    bnd_val = float(np.mean(grad[bnd_band])) if bnd_band.any() else 0.0
    bg_val = float(np.mean(grad[bg_band])) if bg_band.any() else 0.0
    return bnd_val - bg_val


def compute_boundary_contrast(df: pd.DataFrame, max_items_per_ds: int = 300) -> pd.DataFrame:
    rows = []
    for ds, g in df.groupby('dataset'):
        gg = g.sample(min(max_items_per_ds, len(g)), random_state=0)
        for r in tqdm(gg.itertuples(index=False), total=len(gg), desc=f'contrast {ds}'):
            img = read_rgb(Path(r.image_path))
            mask = read_mask(r.mask_path)
            if mask is None:
                continue
            c = boundary_contrast_proxy(img, mask, r=3)
            rows.append({'dataset': ds, 'image_id': r.image_id, 'boundary_contrast_proxy': c})
    return pd.DataFrame(rows)


contrast_df = compute_boundary_contrast(data_index[data_index['has_mask']].copy(), max_items_per_ds=250)
display(contrast_df.groupby('dataset')['boundary_contrast_proxy'].describe())

plt.figure()
for ds, g in contrast_df.groupby('dataset'):
    plt.hist(g['boundary_contrast_proxy'], bins=50, alpha=0.4, label=ds)
plt.xlabel('Boundary contrast proxy (Sobel boundary band minus bg band)')
plt.ylabel('Count')
plt.legend()
plt.title('Boundary contrast proxy distribution')
plt.show()


### 6.1 Visualize low-contrast boundary examples


In [ ]:
def show_low_contrast_examples(k: int = 12):
    merged = contrast_df.merge(data_index[['dataset','image_id','image_path','mask_path']], on=['dataset','image_id'], how='left')
    hard = merged.sort_values('boundary_contrast_proxy', ascending=True).head(k)
    for r in hard.itertuples(index=False):
        img = read_rgb(Path(r.image_path))
        mask = read_mask(r.mask_path)
        show_overlay(img, mask, title=f'LOW boundary contrast | {r.dataset} | {r.image_id} | proxy={r.boundary_contrast_proxy:.4f}')


show_low_contrast_examples(k=6)


## 7. Save summary tables

Xuất ra CSV để đưa vào report/slide.


In [ ]:
out_dir = Path('./eda_outputs')
out_dir.mkdir(parents=True, exist_ok=True)

stats.to_csv(out_dir/'mask_stats.csv', index=False)
feat_df.to_csv(out_dir/'domain_features.csv', index=False)
contrast_df.to_csv(out_dir/'boundary_contrast.csv', index=False)

print('Saved to', out_dir.resolve())
